In [18]:
from pathlib import Path
import numpy as np
import json
import dataset
from datetime import datetime

In [11]:
PROPOSAL_DIR = Path('/mnt/maxiv/biomax/20180479/20181119/')
SAMPLE_DIR = PROPOSAL_DIR / 'raw' / 'Sample-4-11'
SNAPSHOT_DIR = 'timed_snapshots'
TIME_OFFSET = 102.19728493690491

In [129]:
from IPython.core.debugger import set_trace

def read_meta(filename) -> dict:
    with open(filename, 'r') as f:
        return json.load(f)
    
def get_meta_files(snapshot_dir):
    return Path(snapshot_dir).glob('*.meta.txt')

def fts(ts):
    return datetime.fromtimestamp(ts).strftime('%H-%M-%S.%f')

class QueueEntry:
    def __init__(self, meta, sample_dir):
        self.meta = meta
        self.sample_dir = Path(sample_dir)
        self.master_file = Path(meta['fileinfo']['filename'])  # /data/visitors/...
        self.nbr_images = meta['oscillation_sequence'][0]['number_of_images']
        self.zoom = meta.get('zoom1', None)
        self.frontlight = meta.get('backlight', None)
        self.prefix = f"{meta['fileinfo']['prefix']}_{meta['fileinfo']['run_number']}"
        
    # TODO: Export found images with time corrected timestamp (synchronized after hdf5 data)
    def find_closest_images(self):
        THRESHOLD = 0.1
        images = []
        found_ts = []
        timestamps = dataset.get_timestamps(self.__local_master_file(), self.nbr_images)
        candidates = sorted((self.sample_dir / SNAPSHOT_DIR).glob(f'{self.prefix}*.jpeg'), key=lambda p: p.name)
        ts_i = 0
        i = 0
        while i < len(candidates) + 1:
            # We found matches for all timestamps
            if ts_i == len(timestamps):
                break
            ts = timestamps[ts_i]
            img_ts = np.array([float(candidates[i].stem[len(self.prefix) + 1:]),
                      float(candidates[i + 1].stem[len(self.prefix) + 1:])]) - TIME_OFFSET
            
            # Skip images taken before collection
            if ts > img_ts[1]:
                i += 2
                continue
                
            diff = np.abs(img_ts - ts)
            closest_idx = np.argmin(diff)
             
            # Only add images if they are a taken close enough
            if diff[closest_idx] < THRESHOLD:
                images.append(candidates[i + closest_idx])
                found_ts.append(ts)
                ts_i += 1
                i += 2
                print(f'match! {fts(img_ts[closest_idx])} ~= {fts(ts)}')
            else:  # Skip frame if there is not image taken close enough
                ts_i +=1
                continue
            
        return images, found_ts
            
            
                

    def __local_master_file(self):
        return self.sample_dir / self.master_file.name
        
    def __str__(self):
        return str(Path(self.master_file))
    
    def __repr__(self):
        return self.__str__()
    
entries = [QueueEntry(read_meta(meta_file), SAMPLE_DIR) for meta_file in get_meta_files(SAMPLE_DIR / SNAPSHOT_DIR)]

In [130]:
e = entries[0]
res = list(zip(*e.find_closest_images()))
print(len(res))
[print(fts(float(i.stem[13:]) - TIME_OFFSET), fts(ts)) for i, ts in res]

match! 22-43-15.022715 ~= 22-43-15.022715
match! 22-43-15.212715 ~= 22-43-15.222721
match! 22-43-15.472715 ~= 22-43-15.422727
match! 22-43-15.832715 ~= 22-43-15.822739
match! 22-43-16.122715 ~= 22-43-16.022745
match! 22-43-16.382715 ~= 22-43-16.422757
match! 22-43-16.602715 ~= 22-43-16.622763
match! 22-43-16.892715 ~= 22-43-16.822769
match! 22-43-17.272715 ~= 22-43-17.222781
match! 22-43-17.362715 ~= 22-43-17.422787
match! 22-43-17.602715 ~= 22-43-17.622793
match! 22-43-17.852715 ~= 22-43-17.822799
match! 22-43-18.082715 ~= 22-43-18.022805
match! 22-43-18.492715 ~= 22-43-18.422817
match! 22-43-18.752715 ~= 22-43-18.822829
match! 22-43-19.042715 ~= 22-43-19.022835
match! 22-43-19.282715 ~= 22-43-19.222841
match! 22-43-19.682715 ~= 22-43-19.622853
match! 22-43-19.842715 ~= 22-43-19.822859
match! 22-43-20.002715 ~= 22-43-20.022865
match! 22-43-20.222715 ~= 22-43-20.222871
match! 22-43-20.372715 ~= 22-43-20.422877
match! 22-43-20.652715 ~= 22-43-20.622883
match! 22-43-20.892715 ~= 22-43-20

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
%reset